In [1]:
import cv2

import numpy as np

import sys
# import cv2 as cv
%matplotlib qt4
%matplotlib auto
import matplotlib.pyplot as plt

sys.path.append('/home/gravishlab/Documents/Python/')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/Tracker/')

from pyqtgraph.Qt import QtCore, QtGui
# QtGui.QApplication.instance().exec_()

from Tracker import Tracker

Using matplotlib backend: Qt4Agg


In [19]:
%qtconsole

In [3]:
vid_locations = '/media/gravishlab/AE3ABE353ABDFB03/Users/GravishLab/AntTrack/OldVideos/'
vid = vid_locations + 'RunningAntFTIR.avi'
video = Tracker.Tracker(vid,min_object_size=500, verbose='False')
# cv2.VideoCapture(vid)
print('done')

done


In [4]:
# video.bkg_sep = 20
video.threshold_val = 0.2 # CLEAR how diff from background (1) will identify an object
# video.threshold_val = 0.15 #  GREEN how diff from background (1) will identify an object

video.load_video()
video.compute_background()          # form background image
video.remove_background()           # remove background
video.threshold()                   # threshold to segment features
video.find_distance()               # remove dist before closing on clear bridge
video.morpho_closing()
video.find_objects()
video.draw_contours()
# video.find_distance()               # takes, dist tranform, finds peaks, associates to ants, finds head/gaster


# video.find_skeleton()
# video.draw_skeletons()
# video.save_JSON() ## CURRENTLY WON'T SAVE SKEL STUFF
# print(video.contours)
print('done')

Loading
0
100
200
300
400
500
all_loaded!
BKG
NORM
Threshold
Distance Transform
Morphological closing
Tracking
done


In [129]:
# TEST ANALYSIS VARIABLES FOR RANGE OF FRAMES
thresh = 0.2
minsize = 500
maxsize = 1800
plt.close('all')

# Frames=list(range(400,421))
Frames=list(range(420,421))#[0::5]
plt.figure(figsize=(12,5))
plt.ion()


pixOI = [116+270,77+340]
pixOI2 = [116+268,77+340]

for Frame in Frames:
    n_splt=2
#     ROI = [300:500,300:500]

    
    
    raw_gray = np.repeat(video.frames[Frame].copy()[:,:,np.newaxis],3, axis=2)
    ax1=plt.subplot(1,n_splt,1)
    plt.imshow(raw_gray)
    plt.ylim((270,470))
    plt.xlim((340,540))
    ax1.set_title('Raw video')
    plt.axis('off')
    plt.scatter(pixOI[1],pixOI[0],s =2, c = "r", linewidths=1)
    plt.scatter(pixOI2[1],pixOI2[0],s =2, c = "g", linewidths=1)
    plt.gca().invert_yaxis()
    
    bkgd = video.frames_normed[Frame].copy()[270:470,340:540]
#     bkgd_norm = ((bkgd-np.min(bkgd))*
#                  (255/(np.max(bkgd)-np.min(bkgd)) ))
#     bkgd_norm2 = np.zeros(bkgd.shape)
#     bkgd_norm2 = cv2.normalize(bkgd, bkgd_norm2, 0, 255, cv2.NORM_MINMAX)
    bkgd_norm = (128/(np.max(bkgd-1)))*(bkgd-1) + 128
    bkgd_gray = np.repeat(bkgd_norm[:,:,np.newaxis],3, axis=2)
    ax2=plt.subplot(1,n_splt,2)
    ax2.clear()
    plt.imshow(bkgd)
#     plt.imshow(bkgd_norm)
    ax2.set_title('Background Div')
    plt.axis('off')
    plt.text(20,40,Frame)
    
#     # take threshold
#     video.threshold_val = thresh
#     temp = (video.frames_normed[Frame] < 1 - video.threshold_val) | (video.frames_normed[Frame] > 1 + video.threshold_val)
#     ax3=plt.subplot(n_splt,1,3)
#     plt.imshow(temp)
#     ax3.set_title('Thresholded')
#     plt.axis('off')
    
#     # remove short distances
#     # cv.DISL_L1 = 4-connected distance (diagonal is 2), cv.DIST_L2 = 8-connected
#     ant_dist = cv2.distanceTransform(np.array(temp, dtype=np.uint8), cv2.DIST_L2, 3)
#     # remove any points with small distance
#     rm_edges = ant_dist.copy()
#     rm_edges[ant_dist < 2] = 0
#     ax4=plt.subplot(n_splt,1,4)
#     plt.imshow(rm_edges)
#     ax4.set_title('Remove dist < 2')
#     plt.axis('off')
    
#     # close image
#     kernel = cv2.getStructuringElement( cv2.MORPH_RECT, (5,5))
#     temp2 = cv2.morphologyEx(np.uint8(rm_edges>0), cv2.MORPH_CLOSE, kernel, iterations = 7) 
# #     ax2=plt.subplot(n_splt,1,5)
# #     plt.imshow(temp2)
    
    
#     plt.sca(ax1)
#     plt.scatter(x,y,s =2, c = "r", linewidths=1)
    plt.pause(1)
    
    
pixOI_val=[fr[pixOI[0],pixOI[1]] for fr in video.frames[350:500]]
pixOI2_val=[fr[pixOI2[0],pixOI2[1]] for fr in video.frames[350:500]]
plt.figure()
plt.plot(pixOI_val,'-r')
plt.plot(pixOI2_val,'-g')